# Text Classification using Keras and 🔭 Galileo

In this tutorial, we'll train a model with Keras and explore the results in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

In [ ]:
#@title Install `dataquality` with `pip install dataquality`

try:
    import dataquality as dq
except ImportError:
    # Upgrade pip
    !pip install -U pip &> /dev/null
    # Install HF datasets for downloading the example datasets
    !pip install -U dataquality datasets evaluate transformers 'tensorflow<2.11.0' &> /dev/null
    
    print('👋 Installed necessary libraries and restarting runtime! This should only need to happen once.')
    print('🙏 Continue with the rest of the notebook or hit "Run All" again!')


# 1. Initialize Galileo

In [ ]:
import dataquality as dq
# 🔭🌕 Initializing a new run in Galileo. Each run goes under a project.
dq.init(task_type="text_classification", 
        project_name="text_classification_keras", 
        run_name=f"example_run_tc_keras_1")

# 2. Load Data

In [ ]:
#@title 🤗 HuggingFace Dataset
#@markdown You can find more datasets [here](https://huggingface.co/datasets?language=language:en&task_categories=task_categories:text-classification&task_ids=task_ids:multi-class-classification&sort=downloads).

dataset_name = "generalization/newsgroups_Full-p_1" #@param ["generalization/newsgroups_Full-p_1", "banking77", "emotion", "rungalileo/conv_intent"] {allow-input: true}
print(f"You selected the {dataset_name} dataset")

from IPython.utils import io
from datasets import load_dataset, get_dataset_config_names

# Try to load the data. If a config (subset) is needed, pick one
try:
  with io.capture_output() as captured:
    data = load_dataset(dataset_name)
except ValueError as e:
  if "Config name is missing" not in repr(e):
    raise e

  configs = get_dataset_config_names(dataset_name)
  print(f"The dataset {dataset_name} has multiple subsets {configs}.")
  config = input(f"🖖 Enter the name of the subset to pick (or leave blank for any): ")
  if config:
    assert config in configs, f"{config} is not a valid subset"
  else:
    config = configs[0]
  with io.capture_output() as captured:
    data = load_dataset(dataset_name, name=config)

# Check that the dataset has at least train and either of validation/test
assert "train" in data and {"validation", "test"}.intersection(data), \
f"💾 The dataset {dataset_name} has either no train, or no validation or test splits, select another one."

print(f"\n🏆 Dataset {dataset_name} loaded succesfully")

# A small function for minimizing the dataset for testing purposes
import os

def _minimize_for_ci() -> bool:
    return os.getenv("MINIMIZE_FOR_CI", "false") == "true"

In [ ]:
#@markdown Convert HF dataset to Pandas dataframes 
import pandas as pd

def load_pandas_df(data):
  # Find the name of the ground truth column
  good_col_names = [name for name in list(data['train'].features) if "label" in name]
  if len(good_col_names) == 1:
    label_col = good_col_names[0]
  else:
    col_names = list(data['train'].features)
    print(f"The name of the columns are {col_names}.")
    label_col = input(f"🏅 Please enter the name of the column containing the labels: ")
    assert label_col in col_names, f"{label_col} is not an existing column"

  # Load the labels in a dictionary
  labels = data['train'].features[label_col].names
  labels = {v:k for v, k in enumerate(labels)}

  # Load the train data into a frame
  train_data = data["train"]
  train_df = pd.DataFrame.from_dict(train_data)
  train_df['label'] = train_df[label_col].map(labels)
  train_df['id'] = train_df.index

  # Load the test data into a frame
  test_split_name = "validation" if "validation" in data else "test"
  test_data = data[test_split_name]
  test_df = pd.DataFrame.from_dict(test_data)
  test_df['label'] = test_df[label_col].map(labels)
  test_df['id'] = test_df.index

  return train_df, test_df

train_df, test_df = load_pandas_df(data)
labels = train_df.label.unique().tolist()

if _minimize_for_ci():
  train_df = train_df[:10]
  labels = train_df.label.unique().tolist()
  test_df = test_df[test_df.label.isin(train_df.label.unique())][:10]

# 3. Log input data with Galileo
Input data can be logged via `log_data_samples` (or `log_dataset` for logging iterables). This step will log input samples, gold labels, data split, and list of all labels. You can achieve this by adding 1 line of code to the standard PyTorch Dataset Class.

In [ ]:
from transformers import AutoTokenizer
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from dataquality.integrations.keras import add_sample_ids

# 🔭🌕 Logging the dataset with Galileo
dq.log_dataset(train_df, split="training")
dq.log_dataset(test_df, split="test")

# 🔭🌕 Registering the list of labels for the run
dq.set_labels_for_run(labels)

# Fix batch size (newsgroups20 requires small batch_size to avoid OOM)
BATCH_SIZE = 4

# Tokenize inputs, get attention mask and create train and test datasets
MAX_LENGTH = max(train_df.text.str.split().apply(len).max(), 
                 test_df.text.str.split().apply(len).max())
MAX_LENGTH = min(MAX_LENGTH, 512)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
datasets = []
for df in [train_df, test_df]:
  inputs = tokenizer(df.text.to_list(), truncation=True, padding = "max_length", max_length=MAX_LENGTH, return_tensors = "np")
  inputs["input_ids"] = add_sample_ids(inputs["input_ids"], df.id.values)

  # Get our label index for model training
  input_labels = to_categorical([labels.index(label) for label in df.label])
  dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), input_labels)).batch(BATCH_SIZE)
  datasets.append(dataset)

train_ds, test_ds = datasets

# 4. Log Keras model data with Galileo

Model data can be logged via a `DataQualityLoggingLayer`. This step will log the model probabilitiy and embeddings. You can achieve this by adding the following line of code to Keras sequential nodel. 

In [ ]:
from dataquality.integrations.keras import DataQualityLoggingLayer
from transformers import TFAutoModel
from tensorflow import keras

def build_model(max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len+1,), dtype=tf.int32, name="input_ids")
    attn_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    input_words = DataQualityLoggingLayer("ids")(input_word_ids) # 🌕🔭 Galileo - log the ids
    
    bert = TFAutoModel.from_pretrained('distilbert-base-uncased')
    bert_outputs = bert(input_ids=input_words, attention_mask=attn_mask)
    last_hidden_states = bert_outputs.last_hidden_state
    clf_output = last_hidden_states[:, 0, :] # CLS token 
    clf_output = DataQualityLoggingLayer("embs")(clf_output) # 🌕🔭 Galileo - log the embeddings
    outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(clf_output)
    outputs = DataQualityLoggingLayer("probs")(outputs) # 🌕🔭 Galileo - log the probabilities

    model = tf.keras.models.Model(inputs=[input_word_ids, attn_mask], outputs=outputs)

    model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(1e-5), 
              metrics=["accuracy"], 
              run_eagerly=True) # 🌕🔭 Galileo - set run_eagerly to True!!

    return model 

# 5. Putting into Action: Training a Model

We complete the training pipeline by using a standard PyTorch training setup. While training, we log the current `epoch` and `split`. To complete logging, we call `dq.finish()` after training.

In [ ]:
from dataquality.integrations.keras import DataQualityCallback

epochs = 2

model = build_model(MAX_LENGTH)

model.fit(train_ds,
          batch_size=BATCH_SIZE, 
          epochs=epochs,
          validation_data=test_ds,
          callbacks=[DataQualityCallback()], # 🌕🔭 Galileo
          ) 

dq.finish() # 🔭🌕 Complete the Galileo workflow with a call to dq.finish()

# General Help and Docs
- To get help with your task's requirements, call `dq.get_data_logger().doc()`
- To see more general data and model logging docs, run `dq.docs()`

In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)